# Bathymetry Subtidal
Tests subtidal bathymetry functions of `bathymetry.py`.

In [1]:
from pathlib import Path
import sys
import time

import ee
import geemap
from geojson import Polygon

sys.path.append(str(Path.cwd().parent.parent))

from eepackages.applications import bathymetry

In [2]:
Map = geemap.Map(center=(52.97, 4.74), zoom=11)
bounds = ee.Geometry(Polygon([[
    (4.574518432617185, 52.91662291147939),
    (4.90548156738281, 52.91662291147939),
    (4.90548156738281, 53.02331125248889),
    (4.574518432617185, 53.02331125248889),
    (4.574518432617185, 52.91662291147939)
]]))
scale = Map.getScale()
# scale = 30
start_date = "2019-01-01"
stop_date = "2021-01-01"

sdb = bathymetry.Bathymetry()
Map.addLayer(bounds)

Enter verification code: 4/1AX4XfWgbdu16sZJ_R3V6TLD6Q6SAssRjgUoxOtbjZiZKTB6uTyVLBKhhRDQ

Successfully saved authorization token.


In [3]:
# Run interactively
image = sdb.compute_inverse_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale*5,
    missions=["L8", "S2"],
    filter_masked=True,
    skip_neighborhood_search=False
)
Map.addLayer(image)# .reproject(ee.Projection('EPSG:3857').atScale(90)))  # without projection, takes default projection
Map

Map(center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

Cleanup tests assets for this notebook

In [22]:
user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
asset_id = f"users/{user_name}/tests/test_inverse_depth"
asset_dict = ee.data.listAssets({"parent": f"projects/earthengine-legacy/assets/users/{user_name}/tests"})
if any([asset["id"] == asset_id for asset in asset_dict["assets"]]):
    ee.data.deleteAsset(asset_id)
ee.data.create_assets(asset_ids=["/".join(asset_id.split("/")[:-1])], asset_type="Folder", mk_parents=True)

Asset users/jaapel/tests already exists.


In [26]:
# Let's use the ee.batch.Export class' static methods to create assets
image = sdb.compute_inverse_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale*5,
    missions=["L8", "S2"],
    filter_masked=True,
    skip_neighborhood_search=False # adds a lot of computation time if False
) #  .reproject(ee.Projection('EPSG:3857').atScale(90))

task = ee.batch.Export.image.toAsset(
    image,
    description="test_compute_inverse_depth",
    region=bounds,
    assetId=asset_id,
    scale=scale,
    crs="EPSG:3857",
    maxPixels=1e11
)

In [27]:
task.start()

In [28]:
task_status = task.status()
while task_status['state'] in ['READY', 'RUNNING']:
    print(task_status)
    time.sleep(30)
    task_status = task.status()
else:
    print(task.status())

{'state': 'READY', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1638870448110, 'update_timestamp_ms': 1638870448110, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GBMMBRZTZW4LUBJQWMDFU54C', 'name': 'projects/earthengine-legacy/operations/GBMMBRZTZW4LUBJQWMDFU54C'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1638870448110, 'update_timestamp_ms': 1638870473521, 'start_timestamp_ms': 1638870473382, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GBMMBRZTZW4LUBJQWMDFU54C', 'name': 'projects/earthengine-legacy/operations/GBMMBRZTZW4LUBJQWMDFU54C'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1638870448110, 'update_timestamp_ms': 1638870473521, 'start_timestamp_ms': 1638870473382, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GBMMBRZTZW4LUBJQWMDFU54C', 'name': 'projects/earthengine-legacy/operations/GBMMBRZTZW4LUBJQWMDFU54C'}
{'state': 'RUNNING', 'descrip

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1638870448110, 'update_timestamp_ms': 1638871146888, 'start_timestamp_ms': 1638870473382, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GBMMBRZTZW4LUBJQWMDFU54C', 'name': 'projects/earthengine-legacy/operations/GBMMBRZTZW4LUBJQWMDFU54C'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1638870448110, 'update_timestamp_ms': 1638871206898, 'start_timestamp_ms': 1638870473382, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GBMMBRZTZW4LUBJQWMDFU54C', 'name': 'projects/earthengine-legacy/operations/GBMMBRZTZW4LUBJQWMDFU54C'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1638870448110, 'update_timestamp_ms': 1638871206898, 'start_timestamp_ms': 1638870473382, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GBMMBRZTZW4LUBJQWMDFU54C', 'name': 'projects/earthengine-legacy/operations/GBMMBRZTZW4LUBJQWMDFU54C'}
{

In [29]:
image_subtidal = ee.Image(asset_id)
Map.addLayer(image_subtidal, {"gamma": 1.2}, 'RGB_raw', True)
Map

Map(bottom=171160.0, center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…